<a href="https://colab.research.google.com/github/seoyeon0905/mimic-readmission-analysis/blob/main/notebooks/06_readmission_model_extension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 04 -> "이 정도로는 안 된다"
- 05 -> "그래도 이런 패턴은 있다"
- 06 -> "그럼 뭘 더 넣으면 좋아질까?"

# 06_readmission_model_extension.ipynb의 목표
- baseline(04) 대비 의미 있는 성능 개선
- features 확장 효과 검증
- 해석 가능한 수준에서 모델 비교

## 06에서 추가하면 좋은 Feature 레벨(우선순위)
### Level1: 입원 정보 확장
이미 df에 있는 것들
- admission_type
- admission_location
- discharge_location
- insurance
- hospital_expire_flag(careful)
### -> 05에서 봤던 risk factor를 모델에 넣는 단계

---

### Level2: 이진/범주 처리
- One-hot encoding
- 결측값 처리 전략 명시

---

### Level3: 모델 2~3개만 비교
- Logistic Regression(확장 버전)
- Tree-based(RandomForest or XGBoost 중 하나)

## 06. Extended Readmission Prediction Model

### Objective
To evaluate whether incorporating additional admission-related features
improves predictive performance compared to the baseline model.

---

## 06. 확장된 재입원 예측 모델

### 목적
본 노트북에서는 baseline 모델에서 사용한 제한적인 변수 구성에
입원 유형, 입원 경로, 퇴원 경로, 보험 유형과 같은
추가적인 입원 관련 변수를 포함함으로써,
30일 이내 ICU 재입원 예측 성능이 얼마나 개선되는지를 평가한다.

본 노트북의 목적은 재입원 예측 성능을 최대화하는 것이 아니라
입원 관련 변수(admission context)를 추가했을 때
예측 성능과 모델 해석이 어떻게 변화하는지를 탐색하는 데 있다.

특히 본 단계에서는
- 표본 수가 매우 제한된 환경에서
- 범주형 변수가 다수 포함될 경우
모델 성능이 반드시 개선되지 않을 수 있음을
의도적으로 확인하는 데 목적이 있다.


본 노트북에서는 baseline 모델과의 공정한 비교를 위해
decision threshold 조정은 수행하지 않았다.


In [ ]:
# colab session setup
%cd /content
!rm -rf mimic-readmission-analysis
!git clone https://github.com/seoyeon0905/mimic-readmission-analysis.git
%cd mimic-readmission-analysis

/content
Cloning into 'mimic-readmission-analysis'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 271 (delta 78), reused 5 (delta 5), pack-reused 155 (from 1)
Receiving objects: 100% (271/271), 634.19 KiB | 13.49 MiB/s, done.
Resolving deltas: 100% (132/132), done.
/content/mimic-readmission-analysis


In [ ]:
# 라이브러리
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import(
    roc_auc_score,
    average_precision_score,
    recall_score,
    confusion_matrix,
    classification_report
)

In [ ]:
# 데이터 로드
df = pd.read_csv("data/processed/first_icu_admissions.csv")
df.shape

(100, 27)

# Feature set
- Baseline set: age, icu_los
- Extended set: age, icu_los, admission_type,
  admission_location, discharge_location, insurance

In [ ]:
# Baseline feature
baseline_features = [
    "age",
    "icu_los"
]

In [ ]:
# Extended feature
extended_features = [
    "age",
    "icu_los",
    "admission_type",
    "admission_location",
    "discharge_location",
    "insurance"
]

In [ ]:
target = "readmitted_30d"

In [ ]:
# hospital_expire_flag는 06에서는 일부러 제외
# 누수 위험
# careful이라고 이미 써놨음

# 전처리
- 범주형 변수에 대해 one-hot encoding 수행
- 결측값은 별도 처리 없이 기존 데이터 기준 유지
- Train/test split은 baseline 모델과 동일한 random_state 사용

In [ ]:
# Train / Test split(baseline과 동일하게)
x = df[extended_features]
y = df[target]

x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [ ]:
# random_state 동일
# stratify=y -> rare event 보호

In [ ]:
# 전처리 파이프라인 구성
# (1) 수치형 / 범주형 나누기
numeric_features = ["age", "icu_los"]
categorical_features = [
    "admission_type",
    "admission_location",
    "discharge_location",
    "insurance"
]

In [ ]:
# (2) ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num","passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

In [ ]:
# 결측값을 일부러 처리 안 하는 이유
# admission-related 변수는 대부분 결측이 의미를 가짐

# 모델
- Logistic Regression (extended)
- RandomForestClassifier (비선형 관계 비교)

## 모델 1: Logistic Regression(Extended)

In [ ]:
# 모델 1: Logistic Regression(Extended)
logreg_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LogisticRegression(
            max_iter=1000,
            class_weight="balanced",
            random_state=42
        ))
    ]
)

In [ ]:
logreg_model.fit(x_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['age', 'icu_los']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['admission_type',
                                                   'admission_location',
                                                   'discharge_location',
                                                   'insurance'])])),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=1000,
                                    random_state=42))])

### 평가 지표
- Recall
- PR-AUC
- AUROC
- Confusion Matrix

In [ ]:
# 모델 1 평가
y_pred = logreg_model.predict(x_test)
y_proba = logreg_model.predict_proba(x_test)[:, 1]

In [ ]:
print("Recall:", recall_score(y_test, y_pred))
print("AUROC:", roc_auc_score(y_test, y_proba))
print("PR-AUC:", average_precision_score(y_test, y_proba))

Recall: 0.0
AUROC: 0.2142857142857143
PR-AUC: 0.06272727272727273


In [ ]:
# AUROC가 baseline 대비 낮아진 것은
# 표본 수가 매우 제한적(n=100, positive=8)이며
# 범주형 변수가 다수 포함된 상황에서 선형 모델이 안정적인 decision boundary를
# 학습하지 못했기 때문으로 해석된다.

# 특히 범주형 변수가 다수 포함된 상황에서 표본 수가 매우 제한적일 경우
# AUROC는 모델의 실제 판별 능력을 과소평가할 수 있다.

In [ ]:
confusion_matrix(y_test, y_pred)

array([[24,  4],
       [ 2,  0]])

In [ ]:
# FP는 늘었는데
# TP는 여전히 0
# 즉, 민감해지지도 않았음

# 확장된 feature를 사용하였음에도 기본 threshold(0.5) 기준에서는
# 재입원 환자를 탐지하지 못했으며
# 이는 feature 추가만으로는 decision boundary를 충분히 이동시키기 어렵다는 점을 보여준다.

# 본 단계에서는 baseline과의 공정한 비교를 위해 threshold 조정은 수행하지 않았다.

In [ ]:
# 여기서 Baseline(4) 결과를 옆에 텍스트로 비교 설명
# recall 0 -> 조금이라도 올라가면 의미 있음
# AUROC 0.48 -> 0.55만 돼도 정보 추가 효과 존재

In [ ]:
# 옵션: ROC / PR curve를 baseline VS extended 겹쳐 그리기

## 모델 2: RandomForest(비선형 비교)

In [ ]:
# 모델 2: RandomForest(비선형 비교)
rf_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", RandomForestClassifier(
            n_estimators=200,
            max_depth=6,
            random_state=42,
            class_weight="balanced"
        ))
    ]
)

In [ ]:
rf_model.fit(x_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['age', 'icu_los']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['admission_type',
                                                   'admission_location',
                                                   'discharge_location',
                                                   'insurance'])])),
                ('model',
                 RandomForestClassifier(class_weight='balanced', max_depth=6,
                                        n_estimators=200, random_state=42))])

In [ ]:
# 모델 2 평가
y_pred_rf = rf_model.predict(x_test)
y_proba_rf = rf_model.predict_proba(x_test)[:, 1]

print("Recall:", recall_score(y_test, y_pred_rf))
print("AUROC:", roc_auc_score(y_test, y_proba_rf))
print("PR-AUC:", average_precision_score(y_test, y_proba_rf))

Recall: 0.0
AUROC: 0.5
PR-AUC: 0.28448275862068967


In [ ]:
# RandomForest 모델 역시 기본 threshold(0.5) 기준에서는
# 재입원 환자를 탐지하지 못했으나
# PR-AUC 상승은 순위 기반 관점에서 양성 사례를
# 상대적으로 상위에 배치하는 경향이
# 개선되었음을 의미한다.

In [ ]:
# RandomForest feature importance 확인
feature_names = (
    numeric_features +
    list(
        rf_model.named_steps["preprocess"]
        .named_transformers_["cat"]
        .get_feature_names_out(categorical_features)
    )
)

importances = rf_model.named_steps["model"].feature_importances_

feat_imp = (
    pd.Series(importances, index=feature_names)
    .sort_values(ascending=False)
)

feat_imp.head(10)

,0
icu_los,0.285789
age,0.237994
discharge_location_SNF,0.098863
discharge_location_DEAD/EXPIRED,0.095963
admission_location_EMERGENCY ROOM ADMIT,0.074460
admission_location_TRANSFER FROM HOSP/EXTRAM,0.041104
discharge_location_HOME HEALTH CARE,0.029506
insurance_Private,0.026121
discharge_location_REHAB/DISTINCT PART HOSP,0.025978
insurance_Medicare,0.023695


In [ ]:
# discharge_location = DEAD/EXPIRED는
# 퇴원 이후 재입원을 직접적으로 설명하기보다는
# 입원 당시 중증도를 반영하는 간접 지표로 해석해야 하며
# 인과적 의미로 해석해서는 안 된다.

In [ ]:
# RandomForest의 feature importance 분석 결과
# 퇴원 경로 및 입원 경로 관련 변수가 상대적으로 높은 중요도를 보였으며
# 이는 재입원 위험이 단순한 임상 중증도뿐 아니라
# 병원 시스템적 요인과도 연관되어 있음을 시사한다.

이는 재입원 위험이
단순한 환자 개인의 임상적 중증도뿐 아니라
퇴원 이후 환자가 놓이게 되는 의료 환경과
병원 시스템적 요인과도 함께 연관되어 있을 가능성을 시사한다.


# 비교 및 해석
- Baseline 대비 성능 개선 여부
- 선형 모델과 트리 모델 간 차이
- 어떤 feature 유형이 재입원 예측에 기여하는지 해석

---

- Baseline 대비:
  - 단순 변수만으로는 거의 예측 불가
  - 입원 관련 feature 추가 시 recall / PR-AUC 개선

- Logistic vs RF:
  - Logistic: 해석 가능, 방향성 확인
  - RF: 비선형 패턴 일부 포착 가능

- 결론:
  - “예측 성능은 여전히 제한적”
  - 하지만 입원 맥락 정보가 재입원과 관련 있음을 확인
  - → 06의 역할 완수

## 해석
- Logistic
  - recall: 0 -> 0(개선 없음)
  - PR-AUC: 0.09 -> 0.06(오히려 감소)

- RandomForest
  - recall: 0 -> 0
  - PR-AUC: 0.09 -> 0.28(개선됨)

즉, 전반적 개산이 아니라
- Logistic: 개선 없음(오히려 성능 저하)
- RF: PR-AUC 기준으로만 개선

- 입원 관련 feature를 추가하였으나 선형 모델(Logistic Regression)에서는 예측 성능 개선이 관찰되지 않았다.
- 반면 RandomForest 모델에서는 PR-AUC가 baseline 대비 상승하여 입원 경로 및 보험과 같은 범주형 변수들이 비선형적으로 재입원 위험과 연관될 가능성을 시사한다.

In [ ]:
# 본 단계에서는 feature 확장의 효과를 개략적으로 검증하는 데 목적이 있으며
# 추가적인 성능 개선을 위한 threshold 최적화,
# 하이퍼파라미터 튜닝
# 교차 검증은 다음 단계에서 수행한다.